# Xavier Deep network version

# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Eager mode on

In [2]:
tf.enable_eager_execution()

# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.4442, train_loss: 2.05249906, train_accuracy: 0.6328, test_Accuracy: 0.3818
Epoch: [ 0] [    1/  468] time: 0.6793, train_loss: 1.93041635, train_accuracy: 0.6172, test_Accuracy: 0.5202
Epoch: [ 0] [    2/  468] time: 0.9141, train_loss: 1.68725586, train_accuracy: 0.7500, test_Accuracy: 0.6194
Epoch: [ 0] [    3/  468] time: 1.1409, train_loss: 1.41266954, train_accuracy: 0.7266, test_Accuracy: 0.6493
Epoch: [ 0] [    4/  468] time: 1.3676, train_loss: 1.18582010, train_accuracy: 0.6953, test_Accuracy: 0.7176
Epoch: [ 0] [    5/  468] time: 1.5918, train_loss: 0.95347941, train_accuracy: 0.7891, test_Accuracy: 0.7844
Epoch: [ 0] [    6/  468] time: 1.8217, train_loss: 0.74819553, train_accuracy: 0.8125, test_Accuracy: 0.7582
Epoch: [ 0] [    7/  468] time: 2.0467, train_loss: 0.61154401, train_accuracy: 0.8281, test_Accuracy: 0.7833
Epoch: [ 0] [    8/  468] time: 2.2675,

Epoch: [ 0] [   74/  468] time: 17.0395, train_loss: 0.21832043, train_accuracy: 0.9219, test_Accuracy: 0.9035
Epoch: [ 0] [   75/  468] time: 17.2529, train_loss: 0.32004800, train_accuracy: 0.8672, test_Accuracy: 0.9204
Epoch: [ 0] [   76/  468] time: 17.4781, train_loss: 0.21903375, train_accuracy: 0.9297, test_Accuracy: 0.9135
Epoch: [ 0] [   77/  468] time: 17.7076, train_loss: 0.26172560, train_accuracy: 0.9062, test_Accuracy: 0.9051
Epoch: [ 0] [   78/  468] time: 17.9462, train_loss: 0.21773836, train_accuracy: 0.9375, test_Accuracy: 0.8978
Epoch: [ 0] [   79/  468] time: 18.1716, train_loss: 0.34637338, train_accuracy: 0.8984, test_Accuracy: 0.9044
Epoch: [ 0] [   80/  468] time: 18.3915, train_loss: 0.24700768, train_accuracy: 0.9453, test_Accuracy: 0.9222
Epoch: [ 0] [   81/  468] time: 18.6069, train_loss: 0.23192921, train_accuracy: 0.9297, test_Accuracy: 0.9335
Epoch: [ 0] [   82/  468] time: 18.8295, train_loss: 0.29736102, train_accuracy: 0.8984, test_Accuracy: 0.9339
E

Epoch: [ 0] [  148/  468] time: 33.4555, train_loss: 0.05204679, train_accuracy: 0.9922, test_Accuracy: 0.9438
Epoch: [ 0] [  149/  468] time: 33.6690, train_loss: 0.25343269, train_accuracy: 0.9219, test_Accuracy: 0.9444
Epoch: [ 0] [  150/  468] time: 33.8874, train_loss: 0.10684416, train_accuracy: 0.9766, test_Accuracy: 0.9460
Epoch: [ 0] [  151/  468] time: 34.1057, train_loss: 0.12944846, train_accuracy: 0.9375, test_Accuracy: 0.9488
Epoch: [ 0] [  152/  468] time: 34.3239, train_loss: 0.17856355, train_accuracy: 0.9531, test_Accuracy: 0.9504
Epoch: [ 0] [  153/  468] time: 34.5476, train_loss: 0.07587519, train_accuracy: 0.9844, test_Accuracy: 0.9508
Epoch: [ 0] [  154/  468] time: 34.7711, train_loss: 0.21152380, train_accuracy: 0.9219, test_Accuracy: 0.9506
Epoch: [ 0] [  155/  468] time: 35.0009, train_loss: 0.15661627, train_accuracy: 0.9609, test_Accuracy: 0.9504
Epoch: [ 0] [  156/  468] time: 35.2231, train_loss: 0.13233268, train_accuracy: 0.9766, test_Accuracy: 0.9475
E

Epoch: [ 0] [  222/  468] time: 49.9938, train_loss: 0.05645273, train_accuracy: 0.9922, test_Accuracy: 0.9576
Epoch: [ 0] [  223/  468] time: 50.2111, train_loss: 0.19242182, train_accuracy: 0.9531, test_Accuracy: 0.9594
Epoch: [ 0] [  224/  468] time: 50.4353, train_loss: 0.10268214, train_accuracy: 0.9688, test_Accuracy: 0.9601
Epoch: [ 0] [  225/  468] time: 50.6588, train_loss: 0.03280109, train_accuracy: 1.0000, test_Accuracy: 0.9589
Epoch: [ 0] [  226/  468] time: 50.8787, train_loss: 0.15803194, train_accuracy: 0.9453, test_Accuracy: 0.9603
Epoch: [ 0] [  227/  468] time: 51.1099, train_loss: 0.15623832, train_accuracy: 0.9688, test_Accuracy: 0.9595
Epoch: [ 0] [  228/  468] time: 51.3344, train_loss: 0.09782127, train_accuracy: 0.9688, test_Accuracy: 0.9603
Epoch: [ 0] [  229/  468] time: 51.5648, train_loss: 0.05671175, train_accuracy: 0.9844, test_Accuracy: 0.9561
Epoch: [ 0] [  230/  468] time: 51.8023, train_loss: 0.22575526, train_accuracy: 0.9375, test_Accuracy: 0.9529
E

Epoch: [ 0] [  296/  468] time: 66.4923, train_loss: 0.06944358, train_accuracy: 0.9766, test_Accuracy: 0.9628
Epoch: [ 0] [  297/  468] time: 66.7184, train_loss: 0.08888323, train_accuracy: 0.9688, test_Accuracy: 0.9627
Epoch: [ 0] [  298/  468] time: 66.9477, train_loss: 0.02686856, train_accuracy: 0.9922, test_Accuracy: 0.9607
Epoch: [ 0] [  299/  468] time: 67.1723, train_loss: 0.06676940, train_accuracy: 0.9688, test_Accuracy: 0.9608
Epoch: [ 0] [  300/  468] time: 67.3905, train_loss: 0.04729158, train_accuracy: 0.9766, test_Accuracy: 0.9622
Epoch: [ 0] [  301/  468] time: 67.6134, train_loss: 0.12113629, train_accuracy: 0.9531, test_Accuracy: 0.9630
Epoch: [ 0] [  302/  468] time: 67.8350, train_loss: 0.10192113, train_accuracy: 0.9688, test_Accuracy: 0.9626
Epoch: [ 0] [  303/  468] time: 68.0565, train_loss: 0.08748785, train_accuracy: 0.9688, test_Accuracy: 0.9637
Epoch: [ 0] [  304/  468] time: 68.2804, train_loss: 0.12438609, train_accuracy: 0.9609, test_Accuracy: 0.9644
E

Epoch: [ 0] [  370/  468] time: 83.0609, train_loss: 0.02590164, train_accuracy: 1.0000, test_Accuracy: 0.9632
Epoch: [ 0] [  371/  468] time: 83.2803, train_loss: 0.06930669, train_accuracy: 0.9766, test_Accuracy: 0.9652
Epoch: [ 0] [  372/  468] time: 83.4997, train_loss: 0.16387059, train_accuracy: 0.9688, test_Accuracy: 0.9664
Epoch: [ 0] [  373/  468] time: 83.7243, train_loss: 0.09610395, train_accuracy: 0.9844, test_Accuracy: 0.9666
Epoch: [ 0] [  374/  468] time: 83.9452, train_loss: 0.06315024, train_accuracy: 0.9766, test_Accuracy: 0.9653
Epoch: [ 0] [  375/  468] time: 84.1671, train_loss: 0.05466352, train_accuracy: 0.9922, test_Accuracy: 0.9636
Epoch: [ 0] [  376/  468] time: 84.3862, train_loss: 0.08280216, train_accuracy: 0.9688, test_Accuracy: 0.9659
Epoch: [ 0] [  377/  468] time: 84.6062, train_loss: 0.15085006, train_accuracy: 0.9688, test_Accuracy: 0.9680
Epoch: [ 0] [  378/  468] time: 84.8266, train_loss: 0.15551591, train_accuracy: 0.9609, test_Accuracy: 0.9692
E

Epoch: [ 0] [  444/  468] time: 99.4326, train_loss: 0.20337307, train_accuracy: 0.9375, test_Accuracy: 0.9675
Epoch: [ 0] [  445/  468] time: 99.6537, train_loss: 0.06113275, train_accuracy: 0.9922, test_Accuracy: 0.9673
Epoch: [ 0] [  446/  468] time: 99.8762, train_loss: 0.04254772, train_accuracy: 1.0000, test_Accuracy: 0.9665
Epoch: [ 0] [  447/  468] time: 100.0923, train_loss: 0.09401506, train_accuracy: 0.9688, test_Accuracy: 0.9655
Epoch: [ 0] [  448/  468] time: 100.3119, train_loss: 0.06421863, train_accuracy: 0.9844, test_Accuracy: 0.9662
Epoch: [ 0] [  449/  468] time: 100.5365, train_loss: 0.07544518, train_accuracy: 0.9766, test_Accuracy: 0.9659
Epoch: [ 0] [  450/  468] time: 100.7660, train_loss: 0.10679516, train_accuracy: 0.9688, test_Accuracy: 0.9657
Epoch: [ 0] [  451/  468] time: 100.9965, train_loss: 0.09860048, train_accuracy: 0.9766, test_Accuracy: 0.9657
Epoch: [ 0] [  452/  468] time: 101.2231, train_loss: 0.02897664, train_accuracy: 0.9922, test_Accuracy: 0.

# Test accuracy : 96.88 %